In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import json
from datetime import datetime

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


# this enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

import config_v2 as cfg

from library_report_v2 import Cleaning as cln
from library_report_v2 import Graphing as grp
from library_report_v2 import Processing as pro
from library_report_v2 import Configuration as repcfg

Baseline en config_v2.py: ['2023-09-15', '2024-04-22']
Study en config_v2.py: ['2024-04-22', '2024-04-29']


In [4]:
def show_response_contents(df):
    print("The response contains:")
    print(json.dumps(list(df['variable'].unique()), sort_keys=True, indent=4))
    print(json.dumps(list(df['device'].unique()), sort_keys=True, indent=4))

In [5]:
df = pd.read_pickle(project_path / 'data' / _PICKLED_DATA_FILENAME)
show_response_contents(df)

The response contains:
[
    "ilu-consumo-activa",
    "front-potencia-activa",
    "tr",
    "consumo-energia-reactiva-capacitiva",
    "kw-tr",
    "consumo-energia-reactiva-inductiva",
    "factor-de-potencia",
    "ilu-potencia-activa",
    "front-consumo-activa",
    "consumo-energia-reactiva-total",
    "aa-consumo-activa",
    "area",
    "aa-potencia-activa"
]
[
    "bc57",
    "bc10",
    "bc35",
    "bc8",
    "bc36",
    "bc01-iluminacion-girardot",
    "bc71",
    "bc---s.a.-valledupar",
    "bc---floridablanca",
    "bc14",
    "bc56",
    "bc18",
    "bc60",
    "bc30",
    "bc45",
    "bc21",
    "bc2",
    "bc12",
    "bc16",
    "bc44",
    "bc-santa-monica",
    "bc40",
    "bc15",
    "bc71-2",
    "bc61",
    "bc33",
    "bc42",
    "bc41",
    "bc-291-las-palmas",
    "bc11",
    "bc-megamall",
    "gateway-1-bc-uc",
    "bc65",
    "bc26",
    "bc32",
    "bc38",
    "bc43",
    "bc31",
    "bc58t",
    "bc---s.a.-granada",
    "bc49",
    "bc27",
    "bc64",
    

In [6]:
df = df.sort_values(by=['variable','datetime'])
df = pro.datetime_attributes(df)

df_bl, df_st = pro.split_into_baseline_and_study(df, baseline=cfg.BASELINE, study=cfg.STUDY, inclusive='left')
df_bl['Periodo'] = 'Baseline'
df_st['Periodo'] = 'Estudio'

past_w = df_bl.loc[cfg.PAST_WEEK[0]:cfg.PAST_WEEK[1]]

In [9]:
df_pa = df.query("variable == 'front-potencia-activa'").copy()
front = df.query("variable == 'front-consumo-activa'").copy()

In [11]:
front_hour = front.groupby(by=["variable"]).resample('1h').sum().round(2).reset_index().set_index('datetime')
front_hour = pro.datetime_attributes(front_hour)

In [17]:
front

,value,variable,device,device_name,hour,day,dow,cont_dow,week,month,year
datetime,,,,,,,,,,,
2023-01-01 00:00:00-05:00,7.08,front-consumo-activa,bc---s.a.-valledupar,BC 524 - S.A. Valledupar,0,1,domingo,6.0,52,1,2023
2023-01-01 00:00:00-05:00,37.19,front-consumo-activa,bc---s.a.-granada,BC 821 GRANADA CALI,0,1,domingo,6.0,52,1,2023
2023-01-01 00:00:00-05:00,3.41,front-consumo-activa,bc-816-1---calle-10,BC 816 - Calle 10,0,1,domingo,6.0,52,1,2023
2023-01-01 00:00:00-05:00,2.72,front-consumo-activa,bc71-2,BC 793 - Profesionales,0,1,domingo,6.0,52,1,2023
2023-01-01 00:00:00-05:00,6.88,front-consumo-activa,bc71,BC 787 - Bocagrande Carrera Tercera,0,1,domingo,6.0,52,1,2023
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-29 00:00:00-05:00,4.42,front-consumo-activa,bc7,BC 253 - Puerta del Norte,0,29,lunes,0.0,18,4,2024
2024-04-29 00:00:00-05:00,6.00,front-consumo-activa,bc6,BC 197 - Guatapuri,0,29,lunes,0.0,18,4,2024
2024-04-29 00:00:00-05:00,4.04,front-consumo-activa,bc4,BC 78 - El Cacique,0,29,lunes,0.0,18,4,2024


In [16]:
front_hour = front.groupby(by=["variable", "device_name"]).resample('1h').mean().round(2).reset_index().set_index('datetime')
front_hour = pro.datetime_attributes(front_hour)

In [30]:
front_hour

value  day      dow  cont_dow  week  year
device_name          hour                                              
BC 044 - Piedecuesta 0     3.073546    1  domingo  6.000000    52  2023
                     1     3.074170    1  domingo  6.041667    52  2023
                     2     3.049419    1  domingo  6.083333    52  2023
                     3     3.008838    1  domingo  6.125000    52  2023
                     4     2.988755    1  domingo  6.166667    52  2023
...                             ...  ...      ...       ...   ...   ...
BC 90 - Megamall     19    2.007261    1  domingo  6.791667    52  2023
                     20    1.917697    1  domingo  6.833333    52  2023
                     21    1.920747    1  domingo  6.875000    52  2023
                     22    1.899357    1  domingo  6.916667    52  2023
                     23    1.914813    1  domingo  6.958333    52  2023

[1224 rows x 6 columns]

In [27]:
front_hour = front.groupby(['device_name', 'hour']).mean().round(2).reset_index()

In [31]:
front_hour = front.groupby(['device_name', 'hour']).agg({'value': 'mean', 'day': 'first', 'dow':'first',
       'cont_dow':'first', 'week':'first', 'year':'first' }).reset_index()

In [ ]:
agrupado_suma = df.groupby(['Grupo1', 'Grupo2']).agg({'Valor': 'sum', 'Año': 'first', 'Día': 'first', 'Mes': 'first'}).reset_index()


In [19]:


# Crear un DataFrame de ejemplo
datos = {
    'Grupo1': ['A', 'A', 'B', 'B', 'A'],
    'Grupo2': ['X', 'Y', 'X', 'Y', 'Y'],
    'Valor': [10, 20, 30, 40, 50]
}

df = pd.DataFrame(datos)

print(df)

  Grupo1 Grupo2  Valor
0      A      X     10
1      A      Y     20
2      B      X     30
3      B      Y     40
4      A      Y     50


In [ ]:
import pandas as pd

# Crear un DataFrame de ejemplo
datos = {
    'Grupo1': ['A', 'A', 'B', 'B', 'A'],
    'Grupo2': ['X', 'Y', 'X', 'Y', 'Y'],
    'Valor': [10, 20, 30, 40, 50]
}

df = pd.DataFrame(datos)

# Mostrar el DataFrame original
print("DataFrame original:")
print(df)

# Agrupar por 'Grupo1' y 'Grupo2' y sumar los valores en la columna 'Valor'
agrupado_suma = df.groupby(['Grupo1', 'Grupo2']).sum().reset_index()

# Mostrar el DataFrame agrupado por suma
print("\nDataFrame agrupado por suma:")
print(agrupado_suma)

# Agrupar por 'Grupo1' y 'Grupo2' y sacar el promedio de los valores en la columna 'Valor'
agrupado_promedio = df.groupby(['Grupo1', 'Grupo2']).mean().reset_index()

# Mostrar el DataFrame agrupado por promedio
print("\nDataFrame agrupado por promedio:")
print(agrupado_promedio)
